In [ ]:
!pip3 install --user tensorflow-gpu
!pip install --user spacy

#### Importing Dependencies

In [77]:
from sklearn import linear_model, model_selection
import pandas as pd
import numpy as np
import spacy
import io

#### Preprocessing

In [42]:
nlp = spacy.load('en')

In [71]:
words2vecs = lambda tokens, n: np.array([nlp(token).vector for token in tokens[:n]]) 

In [39]:
vader_lexicon = pd.read_csv(io.StringIO(open('vader_lexicon.csv', encoding='cp437').read()), names=['token', 'norm', 'score', 'vector'])

In [51]:
tokens = vader_lexicon['token'].values

In [119]:
N = 1000

In [120]:
X = vader_lexicon['token'].values[:N]
y = vader_lexicon['score'].values[:N]

In [121]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y)

#### Modeling

In [122]:
from sklearn import linear_model, model_selection
import spacy

class SentimentInterpolator:
    def __init__(self, model):
        self.model = model()
        self.nlp = spacy.load('en')    
    
    def preprocess(self, tokens):
         return np.array([self.nlp(token).vector for token in tokens]) 
    
    def fit(self, X, y):
        self.model.fit(self.preprocess(X), y)
        return self
        
    def predict(self, X):
        return self.model.predict(self.preprocess(X))

In [123]:
interpolator = SentimentInterpolator(linear_model.SGDRegressor)

In [124]:
interpolator.fit(X_train, y_train)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [126]:
interpolator.predict(['bitcoin is great', 'the S&P 500 is probably amazing', 'index funds are correlating the stock market'])

array([0.48436716, 0.16634598, 0.37125068])